In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from typing import Any
from google.oauth2 import service_account
import logging


In [2]:
credentials = service_account.Credentials.from_service_account_file("connection-123-892e002c2def.json")

In [6]:
def calculate_bollinger_bands(credentials) -> pd.DataFrame:
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    query = """
    SELECT
        timestamp,
        price
    FROM
        `connection-123.signals.bitcoin_price`
    WHERE
        DATE(timestamp) < DATE(CURRENT_TIMESTAMP())
    ORDER BY
        timestamp ASC
    """

    query_job = client.query(query)
    results = query_job.result()
    df = results.to_dataframe()

    # Calculate 20-period Simple Moving Average (Middle Band)
    df['middle_band'] = df['price'].rolling(window=20).mean()

    # Calculate 20-period Standard Deviation
    df['std_dev'] = df['price'].rolling(window=20).std()

    # Calculate Upper and Lower Bands (2 standard deviations)
    df['upper_band'] = df['middle_band'] + (2 * df['std_dev'])
    df['lower_band'] = df['middle_band'] - (2 * df['std_dev'])

    # Calculate Bollinger Band Width (optional indicator)
    df['bb_width'] = df['upper_band'] - df['lower_band']

    # Calculate %B (position within bands, optional)
    df['percent_b'] = (df['price'] - df['lower_band']) / (df['upper_band'] - df['lower_band'])

    # Return Bollinger Bands components with timestamp
    bollinger_result = df[['timestamp', 'price', 'middle_band', 'upper_band', 'lower_band', 'bb_width', 'percent_b']].copy()

    return bollinger_result


In [7]:
data = calculate_bollinger_bands(credentials)

In [8]:
print(data)

      timestamp          price    middle_band     upper_band     lower_band  \
0    2024-08-12   58804.234500            NaN            NaN            NaN   
1    2024-08-13   59350.074333            NaN            NaN            NaN   
2    2024-08-14   60601.223178            NaN            NaN            NaN   
3    2024-08-15   58739.193822            NaN            NaN            NaN   
4    2024-08-16   57624.116929            NaN            NaN            NaN   
..          ...            ...            ...            ...            ...   
361  2025-08-08  117463.474511  116781.532529  120941.122803  112621.942255   
362  2025-08-09  116688.366632  116720.869532  120846.932462  112594.806602   
363  2025-08-10  116510.083932  116683.527688  120802.676016  112564.379360   
364  2025-08-11  119266.925169  116772.750457  121039.426448  112506.074467   
365  2025-08-11  120144.004708  116782.160907  121079.119344  112485.202471   

        bb_width  percent_b  
0            NaN     